# Calculate enhancer、eRNA、EPI for Hela

In [10]:
import pandas as pd
pd.set_option('display.max_columns', None)
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import os
import pyBigWig

In [ ]:
# Fuction define. if eRNA intersect with promoter, return False. else return True
def ifFilter(row):
    try:
        if (row['eRNA_start'] >= row['promoter_start']) & (row['eRNA_start'] <= row['promoter_end']):
            return True
        elif (row['eRNA_end'] >= row['promoter_start']) & (row['eRNA_end'] <= row['promoter_end']):
            return True
        else:
            return False
    except:
        try:
            if (row['enhancer_start'] >= row['promoter_start']) & (row['enhancer_start'] <= row['promoter_end']):
                return True
            elif (row['enhancer_end'] >= row['promoter_start']) & (row['enhancer_end'] <= row['promoter_end']):
                return True
            else:
                return False
        except:
            raise KeyError

# enhancer identification

In [ ]:
# sequence data download
cmd = f"bash download.sh"
subprocess.run(cmd, shell=True)

In [ ]:
# Histone modification ChIP-seq call peak
files = ['H3K4me1_r2.bam', 'H3K4me1_r1.bam', 'H3K27ac_r2.bam', 'H3K27ac_r1.bam']
for file in files:
    inputFile = 'control_r1.bam' if 'r1' in file else 'control_r2.bam'
    cmd = f"sicer -t {file} -c {inputFile} -s hg19 -fdr 0.05 -cpu 20 -o ."
    subprocess.run(cmd, shell=True)

# Get standard-format broadPeak file from sicer result
files = [x for x in os.listdir() if (x.endswith('-W200-G600.scoreisland') and 'H3K' in x)]
for file in files:
    df = pd.read_csv(file, sep='\t', header=None)
    df.insert(3, 'peakName', [f'peak{x}' for x in range(1, df.shape[0]+1)])
    df['strand'] = '.'
    df['signalValue'] = df[3]
    df['pValue'] = df[3]
    df['qValue'] = df[3]
    df.to_csv(file.replace('-W200-G600.scoreisland', '.broadPeak'), sep='\t', header=None, index=False)

# Use idr to combine different replicates
files = [x for x in os.listdir() if (x.endswith('.broadPeak') and 'H3K' in x and 'r1' in x)]
for file in files:
    file_rep2 = file.replace('r1', 'r2')
    outfile = file.replace('r1',  'idr')
    cmd = f'''idr --samples {file} {file_rep2} --input-file-type broadPeak \
        --rank q.value --output-file {outfile} --output-file-type broadPeak \
            --plot --log-output-file {outfile}.log'''
    subprocess.run(cmd, shell=True)

# Filter idr result with threshold as 0.1
files = [x for x in os.listdir() if (x.endswith('.broadPeak') and 'H3K' in x and 'idr' in x)]
for file in files:
    df = pd.read_csv(file, sep='\t', header=None)
    df = df[df[4]> int(-125 *  np.log2(0.1))]
    df.to_csv(file.replace('.broadPeak', '_filter.broadPeak'), sep='\t', header=None, index=False)

# H3K4me1 H3K27ac intersection calculation
H3K4me1_file = [x for x in os.listdir() if (x.endswith('_filter.broadPeak') and 'H3K4me1' in x and 'idr' in x)][0]
H3K27ac_file = [x for x in os.listdir() if (x.endswith('_filter.broadPeak') and 'H3K27ac' in x and 'idr' in x)][0]
cmd = f"bedtools intersect -a {H3K4me1_file} -b {H3K27ac_file} > H3K4me1_H3K27ac_intersect.bed"
subprocess.run(cmd, shell=True)

# Retain H3K4me1 H3K27ac intersection part which show lower H3K4me3 histone modification signal. Get primary enhancer.
H3K4me1_bam = [x for x in os.listdir() if (x.endswith('.bam') and 'H3K4me1' in x)]
for file in H3K4me1_bam:
    if f"{file}.bai" in os.listdir():
        continue
    cmd = f"samtools index -@ 20 {file}"
    subprocess.run(cmd, shell=True)
cmd = f'''bamCompare -b1 {H3K4me1_bam[0]} -b2 {H3K4me1_bam[1]} --outFileName H3K4me1_all.bw \
          --normalizeUsing CPM --binSize 10 --operation add --numberOfProcessors 20 --scaleFactorsMethod None'''
subprocess.run(cmd, shell=True)
H3K4me3_bam = [x for x in os.listdir() if (x.endswith('.bam') and 'H3K4me3' in x)]
for file in H3K4me3_bam:
    if f"{file}.bai" in os.listdir():
        continue
    cmd = f"samtools index -@ 20 {file}"
    subprocess.run(cmd, shell=True)
cmd = f'''bamCompare -b1 {H3K4me3_bam[0]} -b2 {H3K4me3_bam[1]} --outFileName H3K4me3_all.bw \
            --normalizeUsing CPM --binSize 10 --operation add --numberOfProcessors 20 --scaleFactorsMethod None'''
subprocess.run(cmd, shell=True)
me1BW = pyBigWig.open('H3K4me1_all.bw')
me3BW = pyBigWig.open('H3K4me3_all.bw')
def ifRetain(row, me1BW, me3BW, me1ScoreAll=1, me3ScoreAll=1):
    chr = row[0]
    start = row[1]
    end = row[2]
    me1Score = np.array(me1BW.stats(chr, start, end, type='sum'))[0] / me1ScoreAll * pow(10, 6)
    me3Score = np.array(me3BW.stats(chr, start, end, type='sum'))[0] / me3ScoreAll * pow(10, 6)
    if me1Score > 1.5 * me3Score:
        return True, me1Score, me3Score
    else:
        return False, me1Score, me3Score
df = pd.read_csv('H3K4me1_H3K27ac_intersect.bed', sep='\t', header=None)
df['result'] = df.apply(ifRetain, axis=1, args=(me1BW, me3BW))
df[['retain', 'me1Score', 'me3Score']] = df['result'].apply(pd.Series)
me1BW.close()
me3BW.close()
df[df['retain']][[0,1,2]].to_csv('enhancer_primary.bed', sep='\t', header=None, index=False)

# DNase-seq data process to get broadPeak.
files = [x for x in os.listdir() if ((x.endswith('.bam')) and ('DNase' in x))]
for file in files:
    cmd = f"sicer -t {file} -s hg19 -fdr 0.05 -cpu 20 -o ."
    subprocess.run(cmd, shell=True)
files = [x for x in os.listdir() if (x.endswith('-W200-G600.scoreisland') and 'DNase' in x)]
for file in files:
    df = pd.read_csv(file, sep='\t', header=None)
    df.insert(3, 'peakName', [f'peak{x}' for x in range(1, df.shape[0]+1)])
    df['strand'] = '.'
    df['signalValue'] = df[3]
    df['pValue'] = df[3]
    df['qValue'] = df[3]
    df.to_csv(file.replace('-W200-G600.scoreisland', '.broadPeak'), sep='\t', header=None, index=False)

# Filter primary enhancer with its DNase broadPeak. Obtain final enhancer.
DNase_file = [x for x in os.listdir() if (x.endswith('.broadPeak') and 'DNase' in x)][0]
cmd = f"bedtools intersect -a enhancer_primary.bed -b {DNase_file} -u -f 0.5 -F 0.5 -e > enhancer.bed"
subprocess.run(cmd, shell=True)

# eRNA identification

In [ ]:
# GROseq data process

# Use Fastq-dump to convert SRA data to fastq data.
singleSraList = [x for x in os.listdir() if x.endswith('single.sra')]
for singleSra in singleSraList:
    cmd = f"parallel-fastq-dump --sra-id {singleSra} -t 20 -O . --gzip"
    subprocess.run(cmd, shell=True)

# Use trim-galore to cut adapter and filter low quality reads.
files = [x for x in os.listdir() if x.endswith('single.fastq.gz')]
for file in files:
    cmd = f"trim_galore -q 20 --max_n 4 --phred33 --stringency 3 -j 4 -o . {file}"
    subprocess.run(cmd, shell=True)

# Use bwa mem to align filtered reads with genome. Remove duplication with SAMtools.
files = [x for x in os.listdir() if x.endswith('_trimmed.fq.gz')]
refGene = '/home/limh/Reference_Genome/Homo/hg19/BWA/genome.fa'
for file in files:
    outfile = file.replace('_single_trimmed.fq.gz', '_sort.bam')
    outfile_rmdup = file.replace('_single_trimmed.fq.gz', '_sort_rmdup.bam')
    cmd = f"bwa mem -t 20 -M {refGene} {file} |samtools view -@ 1 -q 10 -hb -F 3852 -S |samtools sort -@ 10 -o {outfile} -"
    subprocess.run(cmd, shell=True)
    cmd = f"samtools rmdup -s {outfile} {outfile_rmdup}"
    subprocess.run(cmd, shell=True)

# Call peak with Homer
fileList = [x for x in os.listdir() if (x.endswith('_sort_rmdup.bam') and 'GROseq' in x)]
bamFiles = '\t'.join([x for x in fileList])
cmd = f"makeTagDirectory GROseq_Hela/ -genome /home/limh/Reference_Genome/Homo/hg19/genome.fa -checkGC {bamFiles}"
subprocess.run(cmd, shell=True)
folderList = ['GROseq_Hela']
for folder in folderList:
    cmd = f'''findPeaks {folder} -style groseq -o auto -tssSize 250 -minBodySize 250 \
              -tssFold 4 -bodyFold 3 -fragLength 50 -pseudoCount 1 -confPvalue 1.00e-05'''
    subprocess.run(cmd, shell=True)

In [6]:
# eRNA identification

# get the nascent transcripts identifyied by Homer from GRO-seq
df = pd.read_csv('/home/limh/epSameTransposon/Homo/eRnaHela/GROseq_Hela/transcripts.gtf', sep='\t', header=None)
df[[0,3,4,7,5,6]].to_csv('transcripts.bed', sep='\t', header=None, index=False)

# Get the nascent transcripts lies in enhancer region.
cmd = f"bedtools intersect -a transcripts.bed -b enhancer.bed -u -f 0.8 > transcripts_InEnhancer.bed"
subprocess.run(cmd, shell=True)
cmd = f"bedtools merge -s -d -250 -c 4,5,6 -o first,mean,first -i transcripts_InEnhancer.bed > eRNA_primary.bed"
subprocess.run(cmd, shell=True)

# Retrain the nascent transcripts which shows proper length: 500bp ~ 5000bp.
df = pd.read_csv('eRNA_primary.bed', sep='\t', header=None)
df = df[(df[2]-df[1]).between(200, 5000)]
df[3] = [f"eRNA{x}" for x in range(1, len(df)+1)]
df.to_csv('eRNA.bed', sep='\t', header=None, index=False)


# EPI construction

In [ ]:
# eRNA promoter interaction construction
df = pd.read_csv('GSE63525_HeLa_Arrowhead_domainlist.txt', sep='\t')
df['chr1'] = 'chr' + df['chr1']
df[['chr1', 'x1', 'x2']].to_csv('TAD.bed', sep='\t', header=None, index=None)
cmd = f"bedtools intersect -a promoter.bed -b TAD.bed -wo -f 0.8 -F 0.8 -e > promoter_TAD_intersectWo.txt"
subprocess.run(cmd, shell=True)
cmd = f"bedtools intersect -a eRNA.bed -b TAD.bed -wo -f 0.8 -F 0.8 -e > eRNA_TAD_intersectWo.txt"
subprocess.run(cmd, shell=True)
promoter_TAD_intersectWo_df = pd.read_csv('promoter_TAD_intersectWo.txt', sep='\t', header=None)
promoter_TAD_intersectWo_df.columns = ['promoter_chr', 'promoter_start', 'promoter_end', 'promoter_name', 'promoter_score', 'promoter_strand',
                                       'TAD_chr', 'TAD_start', 'TAD_end', 'intersect']
eRNA_TAD_intersectWo_df = pd.read_csv('eRNA_TAD_intersectWo.txt', sep='\t', header=None)
eRNA_TAD_intersectWo_df.columns = ['eRNA_chr', 'eRNA_start', 'eRNA_end', 'eRNA_name', 'eRNA_score', 'eRNA_strand',
                                   'TAD_chr', 'TAD_start', 'TAD_end', 'intersect']
epi_df = pd.merge(eRNA_TAD_intersectWo_df, promoter_TAD_intersectWo_df,
                  on = ['TAD_chr', 'TAD_start', 'TAD_end'], how='inner')

# EPI filter
epi_df['filter'] = epi_df.apply(ifFilter, axis=1)
epi_filter_df = epi_df[epi_df['filter']==False]
epi_filter_df['epi_name'] = epi_filter_df['eRNA_name'].str.cat(epi_filter_df['promoter_name'], sep=';')

# save EPI result
epi_filter_df[['eRNA_chr', 'eRNA_start', 'eRNA_end', 'promoter_chr', 'promoter_start', 'promoter_end',
               'epi_name', 'eRNA_score', 'eRNA_strand', 'promoter_strand']].to_csv('epi.bedpe', sep='\t', header=None, index=None)

In [ ]:
# enhancer-promoter interaction construction
cmd = f"bedtools intersect -a enhancer.bed -b TAD.bed -wo -f 0.8 -F 0.8 -e > enhancer_TAD_intersectWo.txt"
subprocess.run(cmd, shell=True)
promoter_TAD_intersectWo_df = pd.read_csv('promoter_TAD_intersectWo.txt', sep='\t', header=None)
promoter_TAD_intersectWo_df.columns = ['promoter_chr', 'promoter_start', 'promoter_end', 'promoter_name', 'promoter_score', 'promoter_strand',
                                       'TAD_chr', 'TAD_start', 'TAD_end', 'intersect']
enhancer_TAD_intersectWo_df = pd.read_csv('enhancer_TAD_intersectWo.txt', sep='\t', header=None)
enhancer_TAD_intersectWo_df.insert(3, 'enhancer_name', [f"enhancer{x}" for x in range(1, len(enhancer_TAD_intersectWo_df)+1)])
enhancer_TAD_intersectWo_df.insert(4, 'enhancer_score', 1)
enhancer_TAD_intersectWo_df.insert(5, 'enhancer_strand', '.')
enhancer_TAD_intersectWo_df.columns = ['enhancer_chr', 'enhancer_start', 'enhancer_end', 'enhancer_name', 'enhancer_score', 'enhancer_strand',
                                   'TAD_chr', 'TAD_start', 'TAD_end', 'intersect']
epi_df = pd.merge(enhancer_TAD_intersectWo_df, promoter_TAD_intersectWo_df,
                  on = ['TAD_chr', 'TAD_start', 'TAD_end'], how='inner')
# EPI过滤及补充信息
epi_df['filter'] = epi_df.apply(ifFilter, axis=1)
epi_filter_df = epi_df[epi_df['filter']==False]
epi_filter_df['epi_name'] = epi_filter_df['enhancer_name'].str.cat(epi_filter_df['promoter_name'], sep=';')

# EPI结果保存
epi_filter_df[['enhancer_chr', 'enhancer_start', 'enhancer_end', 'promoter_chr', 'promoter_start', 'promoter_end',
               'epi_name', 'enhancer_score', 'enhancer_strand', 'promoter_strand']].to_csv('enhancerPI.bedpe', sep='\t', header=None, index=None)